In [5]:
import tensorflow
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import warnings
warnings.filterwarnings('ignore')

In [6]:
data = r'/Users/shubhamghorpade/Data Science class/Class prartice/New_project/CNN_Project/archive/train'
data_gen = ImageDataGenerator(
    rescale=1/255,
    validation_split=0.2,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
image_size = (224,224)
train_generator = data_gen.flow_from_directory(data,
                                              target_size=image_size,
                                              batch_size=32,
                                              class_mode='categorical', # multi class >> category
                                               # classes=['cats','dogs'],
                                              subset='training')

val_generator = data_gen.flow_from_directory(data,
                                              target_size=image_size,
                                              batch_size=32,
                                              class_mode='categorical', # multi class >> category
                                               # classes=['cats','dogs'],
                                              subset='validation')

base_model = ResNet50(include_top=False, weights='imagenet', input_shape=image_size+(3,))
base_model.trainable = False   # Freeze all convo layers
base_model

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.


<Functional name=resnet50, built=True>

In [12]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(7, activation='softmax')
])
model.add(Flatten())

model.add(Dense(units= 128, activation = 'relu'))
model.add(Dropout(0.4))

model.add(Dense(units=7, activation = 'softmax'))
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 7)              │           903 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 7)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,852,814 (90.99 MB)

 Trainable params: 265,102 (1.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [16]:
from tensorflow.keras.optimizers import Adam
base_model.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', save_best_only=True)
]

# Train classifier on top
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    callbacks=callbacks,
    verbose=2
)

Epoch 1/5
718/718 - 969s - 1s/step - accuracy: 0.2249 - loss: 1.8744 - val_accuracy: 0.2513 - val_loss: 1.8276
Epoch 2/5
718/718 - 1116s - 2s/step - accuracy: 0.2481 - loss: 1.8252 - val_accuracy: 0.2513 - val_loss: 1.8135
Epoch 3/5
718/718 - 1150s - 2s/step - accuracy: 0.2479 - loss: 1.8192 - val_accuracy: 0.2513 - val_loss: 1.8107
Epoch 4/5
718/718 - 1368s - 2s/step - accuracy: 0.2479 - loss: 1.8175 - val_accuracy: 0.2513 - val_loss: 1.8100
Epoch 5/5
718/718 - 1049s - 1s/step - accuracy: 0.2500 - loss: 1.8162 - val_accuracy: 0.2513 - val_loss: 1.8098


In [27]:
import cv2
import numpy as np

image_path = r'/Users/shubhamghorpade/Data Science class/Class prartice/New_project/CNN_Project/archive/train/neutral/Training_221103.jpg'
img = cv2.imread(image_path)
img = cv2.resize(img, (224, 224))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)/255
img = np.expand_dims(img, axis=0)
img.shape

(1, 224, 224, 3)

In [28]:
import numpy as np

class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

pred = model.predict(img)  # shape (1, 7)
predicted_class = np.argmax(pred)  # index of max prob
emotion = class_labels[predicted_class]

print("Predicted Emotion:", emotion)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Predicted Emotion: Happy
